In [2]:
pip install langchain openai gradio llama-index-core llama-index-readers-file llama-index-llms-ollama llama-index-embeddings-huggingface vllm

^C
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      Traceback (most recent call last):
        File "c:\Users\dev\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "c:\Users\dev\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\dev\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 118, in get_requires_for_build_wheel
          return hook(config_settings)
                 ^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\dev\AppData\Local\Temp\pip-build-env-4k284zeb\overlay\

  Using cached vllm-0.6.0.tar.gz (1.2 MB)
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


In [18]:
pip install llama-index-core llama-index-readers-file llama-index-llms-ollama llama-index-embeddings-huggingface

   ---------------------------------------- 0.0/762.2 kB ? eta -:--:--
   ---------------------------------------- 762.2/762.2 kB 6.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/199.4 MB ? eta -:--:--
   ---------------------------------------- 2.1/199.4 MB 11.7 MB/s eta 0:00:17
    --------------------------------------- 4.7/199.4 MB 11.9 MB/s eta 0:00:17
   - -------------------------------------- 7.1/199.4 MB 11.8 MB/s eta 0:00:17
   - -------------------------------------- 8.7/199.4 MB 10.5 MB/s eta 0:00:19
   -- ------------------------------------- 11.0/199.4 MB 10.7 MB/s eta 0:00:18
   -- ------------------------------------- 13.6/199.4 MB 11.0 MB/s eta 0:00:17
   --- ------------------------------------ 16.0/199.4 MB 11.1 MB/s eta 0:00:17
   --- ------------------------------------ 18.4/199.4 MB 11.1 MB/s eta 0:00:17
   ---- ----------------------------------- 21.0/199.4 MB 11.1 MB/s eta 0:00:17
   ---- ----------------------------------- 23.1/199.4 MB 11.1

In [3]:
pip install llama-index-llms-openai llama-index-llms-langchain

In [1]:
from langchain.chat_models import ChatAnyscale, ChatOpenAI
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, GPTListIndex, GPTSimpleKeywordTableIndex, PromptHelper, StorageContext, load_index_from_storage
from llama_index.llms.langchain import LangChainLLM
from langchain import OpenAI
import gradio as gr
import sys
import os

c:\Users\dev\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm = LangChainLLM(ChatOpenAI())

C:\Users\dev\AppData\Local\Temp\ipykernel_13136\2122851681.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = LangChainLLM(ChatOpenAI())


In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY");

In [4]:
# 'docs' 디렉터리가 없으면 생성합니다.
if not os.path.exists("docs"):
    os.makedirs("docs")
    print("디렉터리 'docs'가 생성되었습니다. 이 폴더에 문서를 추가하세요.")
else:
    print(("디렉터리 'docs'가 이미 존재합니다."))


디렉터리 'docs'가 이미 존재합니다.


In [5]:
def construct_index(directory_path):
    max_input_size = 4096
    num_outputs = 512
    chunk_overlap_ratio = 0.2  # Changed this to a float ratio
    chunk_size_limit = 600

    # PromptHelper to handle chunking
    prompt_helper = PromptHelper(max_input_size, num_outputs, chunk_overlap_ratio, chunk_size_limit=chunk_size_limit)

    # Initialize the LangChain LLM with OpenAI's chat model
    llm = LangChainLLM(llm=ChatOpenAI(temperature=0.7, model_name="gpt-4o", max_tokens=num_outputs))

    # Load documents using SimpleDirectoryReader
    documents = SimpleDirectoryReader(directory_path).load_data()

    # Create VectorStoreIndex for document search and response generation
    index = VectorStoreIndex.from_documents(documents, llm=llm, prompt_helper=prompt_helper)

    # Save the index to disk
    index.storage_context.persist(persist_dir='index.json')

    return index



In [6]:
def chatbot(input_text):
    # 저장된 인덱스를 로드
    storage_context = StorageContext.from_defaults(persist_dir='index.json')
    index = load_index_from_storage(storage_context)
    
    # 쿼리할 때 사용되는 쿼리 엔진을 생성합니다.
    query_engine = index.as_query_engine()

    # 입력된 텍스트에 대해 쿼리를 실행하고 응답을 얻습니다.
    response = query_engine.query(input_text+"+한국어로 대답을 생성해줘.")
    
    return str(response)  # 응답을 문자열로 변환하여 반환


In [8]:
# Construct the index from a document directory (e.g., "docs")
index = construct_index("docs")

In [13]:
for i in range(1, 31):
    # 사용자 입력을 받아서 챗봇과 상호작용하는 방식으로 변경
    if __name__ == "__main__":
        user_input = """
            아래 정보에 기반하여 서로 다른 문제를 출제해줘.

            ## 문제의 형식
            1. 문제는 발문과 선지로 이루어져 있다.
            2. 선지는 반드시 4개이고 중복 정답은 없다.
            3. 모든 문제는 '모든 답이 맞다.' 혹은 '모든 답이 틀리다.', '정답이 없다.'라는 선지를 가질 수 없다.
            4. 각 선지에 발문의 내용과 크게 상충되거나, 극단적으로 보여지는 내용은 사용하지 않는다.
            5. 위 네가지 형식을 반드시 준수하되, 숫자가 적은 형식이 우위를 가진다.


            ## 문제 예시
            헤어세팅에 있어서 웨이브의 형상에 따라 분류하는 것으로서 크레스트가 너무 약하게 되어 리지가 눈에 잘 띄지 않은 웨이브는?

            1) 버티컬 웨이브    2) 새도우 웨이브    3) 내로우 웨이브    4) 와이드 웨이브
            정답: 새도우 웨이브
        """
        response = chatbot(user_input)
        with open("output.txt", "a", encoding="utf-8") as file:
            # 파일에 문자열 쓰기
            file.write(str(i) + ". " + response)
            file.write("\n\n")  # 줄바꿈 추가

In [10]:
# 사용자 입력을 받아서 챗봇과 상호작용하는 방식으로 변경
if __name__ == "__main__":
    print("GPT 챗봇을 시작합니다. '종료'라고 입력하면 종료됩니다.")
    while True:
        user_input = input("질문을 입력하세요: ")
        if user_input.lower() == "종료":
            print("챗봇을 종료합니다.")
            break
        response = chatbot(user_input)
        print("챗봇 응답:", response)

GPT 챗봇을 시작합니다. '종료'라고 입력하면 종료됩니다.
챗봇 응답: 1. 얼굴을 깨끗하게 하는 과정에서 사용되는 제품은? 1) 클렌징폼 2) 바디로션 3) 샴푸 4) 립밤
2. 피부를 촉촉하고 부드럽게 하는 제품은? 1) 선크림 2) 클렌징워터 3) 스킨로션 4) 헤어젤
3. 피부를 진정시키는 효과가 있는 제품은? 1) 세럼 2) 마스크시트 3) 토너 4) 아이크림
4. 피부에 수분을 공급하는 제품은? 1) 섀도우 2) 파운데이션 3) 에센스 4) 립틴트
5. 피부를 탄력있게 하는 제품은? 1) 립스틱 2) 블러셔 3) 크림 4) 세럼
챗봇을 종료합니다.


In [14]:
# Gradio interface setup
iface = gr.Interface(fn=chatbot,
                     inputs=gr.Textbox(lines=7, label="Enter your text"),
                     outputs="text",
                     title="챗봇")

iface

Gradio Interface for: chatbot
-----------------------------
inputs:
|-<gradio.components.textbox.Textbox object at 0x00000162B389FAD0>
outputs:
|-<gradio.components.textbox.Textbox object at 0x00000162B389C2C0>

In [15]:
# Launch the Gradio interface
iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://e585a383aa56deeed8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [33]:
iface.close()   # 서버 종료

Closing server running on port: 7865
